In [2]:
# This is a Python script designed to run in a Google Colab notebook.
# To use, copy this code into a Colab cell and run it.

# Install required libraries (uncomment if running in Colab)
# !pip install matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Simulation parameters
GRID_SIZE = 32  # 32x32 grid
D = GRID_SIZE * GRID_SIZE  # Ψ dimension (1024)
D_PHI = 2  # Φ dimension (x, y)
K = 4  # Z dimension (x, y, noise1, noise2)
FRAMES = 50  # Number of animation frames
BETA = 0.1  # KL divergence weight
SIGMA = 2.0  # Gaussian blob standard deviation
LEARNING_RATE = 0.05  # For Φ updates

# Initialize environment and agent
def setup():
    global agent_pos, psi, phi
    # Agent starts at a random position
    agent_pos = [np.random.randint(0, GRID_SIZE), np.random.randint(0, GRID_SIZE)]
    # Initial sensory input Ψ: Gaussian blob centered at agent_pos
    psi = generate_psi(agent_pos)
    # Initial Φ: perturbed position
    phi = np.array([agent_pos[0] + np.random.uniform(-5, 5), agent_pos[1] + np.random.uniform(-5, 5)], dtype=float)
    return psi, phi, agent_pos

def generate_psi(pos):
    """Generate sensory input Ψ: Gaussian blob centered at pos."""
    x, y = np.meshgrid(np.arange(GRID_SIZE), np.arange(GRID_SIZE), indexing='ij')
    psi = np.exp(-((x - pos[0])**2 + (y - pos[1])**2) / (2 * SIGMA**2))
    return psi / np.max(psi)  # Normalize to [0, 1]

def sample_z(phi):
    """Sample Z ~ p(Z|Φ): Gaussian centered on Φ."""
    mean = np.concatenate([phi, [0, 0]])  # Z = [x, y, noise1, noise2]
    cov = np.eye(K) * 0.1  # Small variance
    return np.random.multivariate_normal(mean, cov)

def decode_psi(z):
    """Generate predicted Ψ̂ ~ p(Ψ|Z): Gaussian blob centered at Z[0:2]."""
    return generate_psi(z[0:2])

def kl_divergence(z, phi):
    """Approximate KL(p(Z|Φ) || r(Z)) where r(Z) is standard normal."""
    mean = np.concatenate([phi, [0, 0]])
    cov = np.eye(K) * 0.1
    prior_cov = np.eye(K)  # r(Z) ~ N(0, I)
    kl = 0.5 * (np.trace(np.linalg.inv(prior_cov) @ cov) +
                (mean @ mean) - K +
                np.log(np.linalg.det(prior_cov) / np.linalg.det(cov)))
    return kl

def update_phi(z, psi_hat, psi, phi):
    """Update Φ to minimize -log p(Ψ|Z) + β*KL(p(Z|Φ) || r(Z))."""
    # Reconstruction loss: mean squared error
    recon_error = psi - psi_hat
    mse = np.mean(recon_error**2)
    # Approximate gradient of -log p(Ψ|Z) w.r.t. Φ[0:2] (x, y)
    x, y = np.meshgrid(np.arange(GRID_SIZE), np.arange(GRID_SIZE), indexing='ij')
    grad = np.zeros(D_PHI)
    grad[0] = np.sum(recon_error * (x - z[0]) / (SIGMA**2)) / D  # Gradient w.r.t. x
    grad[1] = np.sum(recon_error * (y - z[1]) / (SIGMA**2)) / D  # Gradient w.r.t. y
    # KL gradient (approximated): Φ - Z[0:2] due to Gaussian mean
    kl_grad = z[0:2] - phi  # Simplified for p(Z|Φ) ~ N(Φ, 0.1I)
    total_grad = grad + BETA * kl_grad
    new_phi = phi - LEARNING_RATE * total_grad
    # Clip to grid bounds
    new_phi = np.clip(new_phi, 0, GRID_SIZE-1)
    return new_phi, mse

# Animation update function
def update(frame):
    global agent_pos, psi, phi
    # Move agent randomly
    dx, dy = np.random.choice([-1, 0, 1]), np.random.choice([-1, 0, 1])
    new_pos = [agent_pos[0] + dx, agent_pos[1] + dy]
    new_pos = np.clip(new_pos, 0, GRID_SIZE-1)
    agent_pos = new_pos.astype(int)
    # Update sensory input
    psi = generate_psi(agent_pos)
    # Sample Z and generate predicted Ψ̂
    z = sample_z(phi)
    psi_hat = decode_psi(z)
    # Update Φ
    phi, mse = update_phi(z, psi_hat, psi, phi)

    # Update plots
    ax[0].clear()
    ax[1].clear()
    ax[2].clear()
    ax[0].imshow(psi, cmap='viridis', origin='upper')
    ax[0].plot(agent_pos[1], agent_pos[0], 'r^', markersize=10)
    ax[0].set_title("True Sensory Input (Ψ)")
    ax[1].imshow(psi_hat, cmap='viridis', origin='upper')
    ax[1].plot(z[1], z[0], 'b*', markersize=10)
    ax[1].set_title("Predicted Sensory Input (Ψ̂)")
    ax[2].scatter([phi[1]], [phi[0]], c='green', marker='o', s=100)
    ax[2].set_title(f"Latent Φ (x={phi[0]:.1f}, y={phi[1]:.1f}, MSE={mse:.4f})")
    ax[2].set_xlim(0, GRID_SIZE)
    ax[2].set_ylim(0, GRID_SIZE)
    ax[2].set_xticks([])
    ax[2].set_yticks([])
    for a in ax:
        a.grid(False)
    return ax

# Set up the simulation
psi, phi, agent_pos = setup()

# Create the animation
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ani = FuncAnimation(fig, update, frames=FRAMES, interval=200, repeat=False)
plt.close()  # Prevent duplicate display in Colab
from IPython.display import HTML
HTML(ani.to_jshtml())  # Display animation in Colab